In [37]:
import random
import sys
import multiprocessing
import gensim
import pickle

In [35]:
def parse_playlist_get_sequence(in_line, playlist_sequence):
    song_sequence = []
    contents = in_line.strip().split("\t")
    
    # 解析歌单序列
    for song in contents[1:]:
        try:
            song_id, song_name, artist, popularity = song.split(':::')
            # 暂存歌曲的id
            song_sequence.append(song_id)
        except:
            print('song format error \n')
            
    for i in range(len(song_sequence)):
        # 随机打乱数据 并添加到playlist_sequence数组中
        random.shuffle(song_sequence)
        playlist_sequence.append(song_sequence)
    



def train_song2vec(file_name, out_file):
    #所有歌单序列
    playlist_sequence = []
    #遍历所有歌单
#     index = 0
    with open(file_name, 'r') as f:
        for line in f.readlines():
            parse_playlist_get_sequence(line, playlist_sequence)
#             index += 1
#             if index > 0:
#                 break

    
    #使用word2vec训练
    cores = multiprocessing.cpu_count()
    print('using all '+str(cores)+" cores")
    print ("Training word2vec model...")
    # 将歌曲id(eg:287140) 转化为嵌入模型形式的
    model = gensim.models.Word2Vec(playlist_sequence, size=150, min_count=3, window=7, workers=cores)
    print ("Saving model...")
    model.save(out_file)

In [36]:
file_path = '/Users/JQC/Desktop/recommendation_system_codes/'
song_sequence_file =file_path + "popular.playlist"
model_file = "./song2vec.model"
train_song2vec(song_sequence_file, model_file)

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

song format error 

using all 4 cores
Training word2vec model...
Saving model...


In [39]:
# '287140': '梦不落\t孙燕姿',
song_dic = pickle.load(open(file_path + "popular_song.pkl","rb"), encoding='utf8')
model_str = "./song2vec.model"
model = gensim.models.Word2Vec.load(model_str)

In [43]:
for song in list(song_dic.keys())[:10]:
    print (song, song_dic[song])

287140 梦不落	孙燕姿
445845011 狂想.Rhapsody	冯建宇
4874881 完美情人[Perfect Lover]	183Club
474987186 星辰（电视剧《择天记》片头曲）（Cover 张杰）	黄仁烁
28029940 拥抱的理由	尹熙水
28029946 三个人的错	王菀之
28029947 拥抱的理由	李泰
5357609 미친거니 (inst.)	宋智恩
28029949 我是你的谁	张含韵
31134863 没有用	徐誉滕


In [50]:
for song_id in list(song_dic.keys())[:10]:
#     [('32196439', 0.8277928829193115),
#      ('32451770', 0.7367876768112183),...]
    result_song_list = model.wv.most_similar(song_id)

    print (song_id, song_dic[song_id])
    print ("\n相似歌曲 和 相似度 分别为:")
    for song in result_song_list:
        print ("\t", song_dic[song[0]], song[1])
    print ("\n")

287140 梦不落	孙燕姿

相似歌曲 和 相似度 分别为:
	 潇洒走一回	叶蒨文 0.9894552826881409
	 Sweet Lullaby	DESERT 0.9567523002624512
	 Anthem (Orchestral Version With Choral Introduction)	Vangelis 0.8830297589302063
	 500 miles	Innocence mission 0.8351076245307922
	 There You'll Be	Faith Hill 0.7826096415519714
	 野花	群星 0.7148452401161194
	 老情歌	吕方 0.697977602481842
	 其实不想走	周华健 0.6953861713409424
	 睡在我上铺的兄弟	老狼 0.6864146590232849
	 流浪歌手的情人	老狼 0.6601611375808716


445845011 狂想.Rhapsody	冯建宇

相似歌曲 和 相似度 分别为:
	 莉亚的C3PO	李克勤 0.9981088638305664
	  命运 	二手玫瑰 0.9955780506134033
	 一辈子的情人	栗锦 0.9914852380752563
	 骄傲的少年	王俊凯 0.9884437322616577
	 光之翼	简弘亦 0.9418626427650452
	 美丽的草原我的家	杭盖 0.939497709274292
	 不爱，也是爱我	谢和弦 0.9127543568611145
	 G时行乐	NINETYNINE 99乐团 0.910685122013092
	 前方的世界	YHBOYS 0.8811504244804382
	 成都成都	宋秉洋 0.8343884348869324


4874881 完美情人[Perfect Lover]	183Club

相似歌曲 和 相似度 分别为:
	 野蛮游戏	蔡依林 0.9385574460029602
	 好好爱^0^(心动时刻版)	By2 0.5789374113082886
	 擦肩而过	郑源 0.5120390057563782
	 君がいるだけで (オルゴール)	米米CLUB 0.493420213460922

---------------------------
## 问题
所以我们用word2vec学会了哪些歌曲和哪些歌曲最接近。

我们来思考一些很现实同时又很难解决的问题。比如：
#### 1）冷启动问题
我们经常会遇到冷启动的问题，比如没有任何信息的歌曲，我们如何对它做推荐呢？
* 如果是歌手发行的新歌曲，我们怎么进行推荐呢？
* 如果我听完(并收藏)了一首很冷门的歌，怎么进行推荐呢？

我们知道新歌(或者小众的歌)是非常难和其他的歌关联上的，我们有的信息太少了(很少有用户在它上面发生行为)。

1.1 一种解决办法当然是推荐热门的歌曲，但是其实没从个人兴趣出发，我们知道这并不是最好的办法，并没有太大的卵用。

1.2 我们把问题的粒度放粗一点，用同样的思路，比如一个可考虑的解决方案是，我们把**歌曲**的粒度上升到对应的**歌手**，把刚才的song_list替换成artist_list，重新用word2vec建模，这样我们可以得到和一个歌手最相关(接近)的歌手，再推荐这个歌手最热门的歌曲，相对1.1的方法针对性强一些。

商品 => 品类<br>
品类list => 送到word2vec里面去学习<br>
[上衣，上衣，上衣，牛仔裤，牛仔裤,连衣裙...]

#### 2）用户兴趣预测问题
我们刚才完成的功能，类似酷狗音乐和网易音乐里针对一首歌的**“相似音乐”**，那么问题又来了，如果我们现在要对一个user用这套song2vec的方式推荐，我们怎么做呢？

* 每个人的兴趣都是有时效性的，这意味着说，3年前我喜欢王菲的歌，去年我喜欢五月天的歌，而今年我可能就改摇滚路线，喜欢汪峰的歌了。
* 每一首歌的热度也是不一样的，有一些热门的歌，如果用户能喜欢，当然是首选

那么，我们来做一个粗暴一点点的处理，把这2个维度拉进来，一起来针对一个用户做推荐。

**把每个用户喜欢(收藏)过的歌，沿着时间轴排好，同时由近到远给不同的衰减因子(比如最近一首歌是1，前一首是0.98，再前一首是0.98^2，以此类推...)，同时我们针对不同的歌曲热度，给定不同的推荐因子(比如热度100的是1，热度80的是0.9...)，每一首歌都可以拿回一个song2vec的推荐列表和对应的相似度，对相似度以时间衰减因子和热度权重进行加权，最后的结果排序后，展示给用户。